# Compare to:
## 1. An empirical study on program failures of deep learning jobs, Zhang et al, ICSE20

### Label automatically notebooks that are DL

based on what libraries they use

In [1]:
import pandas as pd
import utils.config as config
import utils.util as util
import numpy as np

# manually labeled and processed
df_mlerr_labels = pd.read_excel(config.path_default.joinpath('Manual_labeing/cluster_sampled_labeled_processed.xlsx'))

df_err_g = pd.read_excel(config.path_github_error_process.joinpath('nberror_g_all_eid_p.xlsx'))
df_err_k = pd.read_excel(config.path_kaggle_error_process.joinpath('nberror_k_eid_p.xlsx'))

df_err_g.drop_duplicates(subset=['eid'], keep='first', inplace = True)
df_err_k.drop_duplicates(subset=['eid'], keep='first', inplace = True)

def get_DL(df_mlerr_labels, save_path = None):
    df_mlerr_labels = pd.merge(df_mlerr_labels, 
                               pd.concat([df_err_g[["eid","lib_alias"]], df_err_k[["eid","lib_alias"]]], ignore_index=True), 
                               on="eid", how="left")
    df_mlerr_labels["is_DLnb"] = df_mlerr_labels.lib_alias.apply(util.lib_alias_isDL)
    print(sum(df_mlerr_labels["is_DLnb"])/df_mlerr_labels.shape[0])
    print(df_mlerr_labels[df_mlerr_labels.is_DLnb==True].fname.nunique()/df_mlerr_labels.fname.nunique())

    df_mlerr_labels = df_mlerr_labels[df_mlerr_labels.is_DLnb==True]
    df_mlerr_labels = df_mlerr_labels.drop(['is_DLnb', 'lib_alias'], axis=1)
    if save_path:
        df_mlerr_labels.to_excel(save_path, index=False, engine='xlsxwriter')
    
get_DL(df_mlerr_labels, save_path = config.path_default.joinpath("Manual_labeing/cluster_sampled_labeled_DL.xlsx"))

path_default=WindowsPath('C:/Users/yirwa29/Downloads/data_jupyter_nbs_empirical')
0.4584450402144772
0.4599447513812155


### Map our labeling to theirs: 

Bug type - Main categories:

    Execution environment: 
        label_root_cause == config.label_root_cause["environment"]
    Data (dueing data preocessing, data integrity is compromised. i.e., corrupt data, unexpected encoding): 
        label_refined_exp_type == ["jsondecodeerror", "unsupported file type (read file)", "incompleteparseerror"]
        ename=="unicodedecodeerror"
    DL specific: 
        label_if_ML_bug == config.label_if_ML_bug["ML bug"]
    General code error: 
        label_if_ML_bug == config.label_if_ML_bug["python bug"]
    

In [1]:
import pandas as pd
import utils.config as config
import numpy as np

df_mlerr_labels_DL = pd.read_excel(config.path_default.joinpath('Manual_labeing/cluster_sampled_labeled_DL.xlsx'))

path_default=WindowsPath('C:/Users/yirwa29/Downloads/data_jupyter_nbs_empirical')


In [2]:
print(df_mlerr_labels_DL.shape[0]/(390+356))
print(df_mlerr_labels_DL[df_mlerr_labels_DL.nb_source==2].shape[0]/390) # GH
print(df_mlerr_labels_DL[df_mlerr_labels_DL.nb_source==1].shape[0]/356) # Kaggle

0.4584450402144772
0.34102564102564104
0.5870786516853933


In [3]:
df_mlerr_labels_DL["Comp1_DLJobs"] = -1
df_mlerr_labels_DL["Comp1_DLJobs"] = np.where(df_mlerr_labels_DL['label_root_cause'].isin(config.label_root_cause["environment setting"]),
                                              "Execution environment",
                                              df_mlerr_labels_DL["Comp1_DLJobs"])
df_mlerr_labels_DL["Comp1_DLJobs"] = np.where((df_mlerr_labels_DL['label_refined_exp_type'].isin(["jsondecodeerror", "unsupported file type (read file)", "incompleteparseerror"])|(df_mlerr_labels_DL['ename']=="unicodedecodeerror"))&(df_mlerr_labels_DL["Comp1_DLJobs"]=="-1"),
                                              "Data",
                                              df_mlerr_labels_DL["Comp1_DLJobs"])
df_mlerr_labels_DL["Comp1_DLJobs"] = np.where(df_mlerr_labels_DL['label_if_ML_bug'].isin(config.label_if_ML_bug["ML bug"])&(df_mlerr_labels_DL["Comp1_DLJobs"]=="-1"),
                                              "DL specific",
                                              df_mlerr_labels_DL["Comp1_DLJobs"])
df_mlerr_labels_DL["Comp1_DLJobs"] = np.where(df_mlerr_labels_DL['label_if_ML_bug'].isin(config.label_if_ML_bug["python bug"])&(df_mlerr_labels_DL["Comp1_DLJobs"]=="-1"),
                                              "General code error",
                                              df_mlerr_labels_DL["Comp1_DLJobs"])

In [4]:
df_mlerr_labels_DL["Comp1_DLJobs"].value_counts()

Comp1_DLJobs
DL specific              205
Execution environment     68
General code error        66
Data                       3
Name: count, dtype: int64

In [21]:
df_mlerr_labels_DL["Comp1_DLJobs"].value_counts()/df_mlerr_labels_DL.shape[0]

Comp1_DLJobs
DL specific              0.599415
Execution environment    0.198830
General code error       0.192982
Data                     0.008772
Name: count, dtype: float64

Bug type - Sub-categories:

DL specific:

    GPU out of memory/CPU out of memory:
        label_refined_exp_type=="out of memory (OOM)"
    Framework API misuse:
        label_root_cause.isin(config.label_root_cause["API"])
    Tensor Mismatch:
        label_refined_exp_type=="tensor shape mismatch"
    Loss NaN: - 
    
    ++ what is the top 3 for us?
    
Execution environment:

    Path not found:
        label_root_cause=="file/path not found or exist"
    Library not found:
        label_root_cause=="module not installed"
    Permission denied:
        label_root_cause=="settings(permission, environment)"
    
    ++ what is the top 1 for us?
    
General code error:

    Illegal argument:
        label_root_cause.isin(config.label_root_cause["API"])
    Type mismatch:
        label_refined_exp_type.isin(config.label_refined_exp_type["type"])
    Key not found:
        label_refined_exp_type.isin(config.label_refined_exp_type["key"])
    ...
    
    ++ what is the top 3 for us?
    
Data:

    Corrupt data:
        label_refined_exp_type == ["unsupported file type (read file)", "incompleteparseerror"]
    Unexpected encoding:
        label_refined_exp_type == "jsondecodeerror" | ename=="unicodedecodeerror"

In [6]:
df_mlerr_labels_DL_dlspecific = df_mlerr_labels_DL[df_mlerr_labels_DL["Comp1_DLJobs"]=="DL specific"]
n_oom = sum(df_mlerr_labels_DL_dlspecific["label_refined_exp_type"]=="out of memory (OOM)")
n_tm = sum(df_mlerr_labels_DL_dlspecific["label_refined_exp_type"]=="tensor shape mismatch")
n_api = sum(df_mlerr_labels_DL_dlspecific["label_root_cause"].isin(config.label_root_cause["API misuse"])&(df_mlerr_labels_DL_dlspecific["label_refined_exp_type"]!="tensor shape mismatch"))

print("GPU/CPU out of memory: ", n_oom/df_mlerr_labels_DL_dlspecific.shape[0], n_oom/df_mlerr_labels_DL.shape[0])
print("Tensor Mismatch: ", n_tm/df_mlerr_labels_DL_dlspecific.shape[0], n_tm/df_mlerr_labels_DL.shape[0])
print("Framework API misuse: ", n_api/df_mlerr_labels_DL_dlspecific.shape[0], n_api/df_mlerr_labels_DL.shape[0])

print("Other: ", 1-(n_oom+n_tm+n_api)/df_mlerr_labels_DL_dlspecific.shape[0], (df_mlerr_labels_DL_dlspecific.shape[0]-(n_oom+n_tm+n_api))/df_mlerr_labels_DL.shape[0])

df_mlerr_labels_DL_dlspecific.label_refined_exp_type.value_counts()
# our top 3 are the same as theirs (tho we label API misuse a bit differently - as root cause)

GPU/CPU out of memory:  0.07804878048780488 0.04678362573099415
Tensor Mismatch:  0.14634146341463414 0.08771929824561403
Framework API misuse:  0.21951219512195122 0.13157894736842105
Other:  0.5560975609756098 0.3333333333333333


label_refined_exp_type
tensor shape mismatch                                 30
variable not found                                    29
wrong arguments to API                                26
out of memory (OOM)                                   16
attributeerror                                        12
module not found                                      12
keyerror                                              10
valueerror - data value violation                      9
runtimeerror                                           8
unsupported broadcast                                  5
cast exception                                         5
valueerror - data range mismatch                       5
indexerror-nd                                          5
valueerror - feature name mismatch                     4
function not found                                     4
indexerror-1d                                          4
typeerror-notcallable                                  3
typeerro

In [10]:
df_mlerr_labels_DL_env = df_mlerr_labels_DL[df_mlerr_labels_DL["Comp1_DLJobs"]=="Execution environment"]
n_pnf = sum((df_mlerr_labels_DL_env["label_root_cause"]=="file/path not found or exist"))
n_lnf = sum(df_mlerr_labels_DL_env["label_root_cause"]=="module not installed")
n_pd = sum(df_mlerr_labels_DL_env["label_root_cause"]=="settings(permission, environment)")

print("Path not found: ", n_pnf/df_mlerr_labels_DL_env.shape[0], n_pnf/df_mlerr_labels_DL.shape[0])
print("Library not found: ", n_lnf/df_mlerr_labels_DL_env.shape[0], n_lnf/df_mlerr_labels_DL.shape[0])
print("Permission denied: ", n_pd/df_mlerr_labels_DL_env.shape[0], n_pd/df_mlerr_labels_DL.shape[0])

print("Other: ", 1-(n_pnf+n_lnf+n_pd)/df_mlerr_labels_DL_env.shape[0], (df_mlerr_labels_DL_env.shape[0]-(n_pnf+n_lnf+n_pd))/df_mlerr_labels_DL.shape[0])

df_mlerr_labels_DL_env.label_root_cause.value_counts()
# top 3 align

Path not found:  0.5147058823529411 0.1023391812865497
Library not found:  0.16176470588235295 0.03216374269005848
Permission denied:  0.14705882352941177 0.029239766081871343
Other:  0.17647058823529416 0.03508771929824561


label_root_cause
file/path not found or exist         35
module not installed                 11
settings(permission, environment)    10
library versions incompatible         7
change of environment                 3
external control (window closed)      2
Name: count, dtype: int64

In [15]:
df_mlerr_labels_DL_py = df_mlerr_labels_DL[df_mlerr_labels_DL["Comp1_DLJobs"]=="General code error"]
n_arg = sum((df_mlerr_labels_DL_py["label_root_cause"].isin(config.label_root_cause["API misuse"])))
n_type = sum((~df_mlerr_labels_DL_py["label_root_cause"].isin(config.label_root_cause["API misuse"]))&
             (df_mlerr_labels_DL_py["label_refined_exp_type"].isin(config.label_refined_exp_type["type error"])))
n_key = sum((~df_mlerr_labels_DL_py["label_root_cause"].isin(config.label_root_cause["API misuse"]))&
            (df_mlerr_labels_DL_py["label_refined_exp_type"].isin(config.label_refined_exp_type["key error"])))
n_rest = sum((~df_mlerr_labels_DL_py["label_root_cause"].isin(config.label_root_cause["API misuse"]))&
             (df_mlerr_labels_DL_py["label_refined_exp_type"].isin(["function not found", "syntaxerror", "zerodivisionerror"]+
                                                                    config.label_refined_exp_type["variable not found"]+
                                                                    config.label_refined_exp_type["attribute error"]+
                                                                    config.label_refined_exp_type["index error"])))
print("Illegal argument: ", n_arg/df_mlerr_labels_DL_py.shape[0], n_arg/df_mlerr_labels_DL.shape[0])
print("Type mismatch: ", n_type/df_mlerr_labels_DL_py.shape[0], n_type/df_mlerr_labels_DL.shape[0])
print("Key not found: ", n_key/df_mlerr_labels_DL_py.shape[0], n_key/df_mlerr_labels_DL.shape[0])
print("The rest: ", n_rest/df_mlerr_labels_DL_py.shape[0], n_rest/df_mlerr_labels_DL.shape[0])

print("Other: ", 1-(n_arg+n_type+n_key+n_rest)/df_mlerr_labels_DL_py.shape[0], (df_mlerr_labels_DL_py.shape[0]-(n_arg+n_type+n_key+n_rest))/df_mlerr_labels_DL.shape[0])

df_mlerr_labels_DL_py.label_refined_exp_type.value_counts()
# name error is the most, then illegal argument

Illegal argument:  0.18181818181818182 0.03508771929824561
Type mismatch:  0.06060606060606061 0.011695906432748537
Key not found:  0.0 0.0
The rest:  0.4090909090909091 0.07894736842105263
Other:  0.3484848484848485 0.06725146198830409


label_refined_exp_type
variable not found                  17
function not found                  11
attributeerror                       6
module not found                     5
wrong arguments to API               5
indexerror-1d                        3
valueerror - data range mismatch     3
class not found                      2
typeerror                            2
syntaxerror                          2
keyerror                             1
requesterror                         1
typeerror-notiterable                1
systemerror                          1
valueerror                           1
filenotfounderror                    1
typeerror-op                         1
indexerror-nd                        1
typeerror-notcallable                1
typeerror-notsubscriptable           1
Name: count, dtype: int64

In [17]:
df_mlerr_labels_DL_data = df_mlerr_labels_DL[df_mlerr_labels_DL["Comp1_DLJobs"]=="Data"]
n_cd = sum((df_mlerr_labels_DL_data["label_refined_exp_type"].isin(["unsupported file type (read file)", "incompleteparseerror"])))
n_en = sum((df_mlerr_labels_DL_data["label_refined_exp_type"]=="jsondecodeerror")|(df_mlerr_labels_DL_data["ename"]=="unicodedecodeerror"))

print("Corrupt data: ", n_cd/df_mlerr_labels_DL_data.shape[0], n_cd/df_mlerr_labels_DL.shape[0])
print("Unexpected encoding: ", n_en/df_mlerr_labels_DL_data.shape[0], n_en/df_mlerr_labels_DL.shape[0])

print("Other: ", 1-(n_cd+n_en)/df_mlerr_labels_DL_data.shape[0], (df_mlerr_labels_DL_data.shape[0]-(n_cd+n_en))/df_mlerr_labels_DL.shape[0])

df_mlerr_labels_DL_data.label_refined_exp_type.value_counts()
# all three cases are corrupted data

Corrupt data:  1.0 0.008771929824561403
Unexpected encoding:  0.0 0.0
Other:  0.0 0.0


label_refined_exp_type
unsupported file type (read file)    3
Name: count, dtype: int64

ML pipeline stages:

    Initialization (30.8):
        label_ML_pipeline: config.label_ML_pipeline["environment setup"]
    Data preprocessing (30):
        label_ML_pipeline: config.label_ML_pipeline["data preparation"], config.label_ML_pipeline["data visualization"]
    Training&validation (15):
        label_ML_pipeline: config.label_ML_pipeline["model construction"], config.label_ML_pipeline["training"]
    Model evaluation (24.3):
        label_ML_pipeline: config.label_ML_pipeline["evaluation/prediction"]


In [22]:
print("Initialization: ", sum((df_mlerr_labels_DL["label_ML_pipeline"].isin(config.label_ML_pipeline["environment setup"])))/df_mlerr_labels_DL.shape[0])
print("Data preprocessing: ", sum((df_mlerr_labels_DL["label_ML_pipeline"].isin(config.label_ML_pipeline["data preparation"]+config.label_ML_pipeline["data visualization"])))/df_mlerr_labels_DL.shape[0])
print("Training&validation: ", sum((df_mlerr_labels_DL["label_ML_pipeline"].isin(config.label_ML_pipeline["model construction"]+config.label_ML_pipeline["training"])))/df_mlerr_labels_DL.shape[0])
print("Model evaluation: ", sum((df_mlerr_labels_DL["label_ML_pipeline"].isin(config.label_ML_pipeline["evaluation/prediction"])))/df_mlerr_labels_DL.shape[0])

Initialization:  0.07602339181286549
Data preprocessing:  0.24853801169590642
Training&validation:  0.43567251461988304
Model evaluation:  0.23976608187134502


## 2. Bug Analysis in Jupyter Notebook Projects: An Empirical Study, De Santana et al, TSEM2024

### Map our labeling to theirs: 

Bug type - Main categories:

    Kernel: -
    Conversion: -
    Portability: -
    Environment and settings (missing libraries, incompatible libraries..) (SO 43%, GH 36%):
        label_root_cause.isin(["module not installed", "library versions incompatible"])
        we also have: 
            change of environment, 
            file/path not found or exist, 
            settings(permission, environment), 
            external control (window closed)
    Connection: -
    Processing (5%, 2%):
        label_root_cause.isin(config.label_root_cause["resources"])
    Cell defect: -
    Implementation (22%, 44%):
        Semantic Error (not crashes): -
        Syntax Error: -
        Data Science lib wrong usage:
            label_root_cause.isin(config.label_root_cause["API"])&label_if_ML_bug.isin(config.label_if_ML_bug["ML bug"])
        Data Science Algorithm Error: 
            label_root_cause.isin(config.label_root_cause["implementation"])&label_if_ML_bug.isin(config.label_if_ML_bug["ML bug"])
     we also have: NB specific, data, library

In [23]:
import pandas as pd
import utils.config as config
import numpy as np

# manually labeled and processed
df_mlerr_labels = pd.read_excel(config.path_default.joinpath('Manual_labeing/cluster_sampled_labeled_processed.xlsx'))
df_mlerr_labels_sum = pd.read_excel(config.path_default.joinpath('Manual_labeing/cluster_sampled_labeled_sum.xlsx'))

In [27]:
print("Environment and settings: ", 
      sum((df_mlerr_labels["label_root_cause"].isin(["module not installed", "library versions incompatible"])))/df_mlerr_labels.shape[0])
print("Processing: ", 
      sum((df_mlerr_labels["label_root_cause"].isin(config.label_root_cause["insufficient resource"])))/df_mlerr_labels.shape[0])

n_impl_api = sum((df_mlerr_labels["label_root_cause"].isin(config.label_root_cause["API misuse"])&df_mlerr_labels["label_if_ML_bug"].isin(config.label_if_ML_bug["ML bug"])))
n_impl_impl = sum((df_mlerr_labels["label_root_cause"].isin(config.label_root_cause["implementation error"])&df_mlerr_labels["label_if_ML_bug"].isin(config.label_if_ML_bug["ML bug"])))
print("Implementation: ", (n_impl_api + n_impl_impl)/df_mlerr_labels.shape[0])
print("    Implementation - Data Science lib wrong usage: ", n_impl_api/df_mlerr_labels.shape[0])
print("    Implementation - Data Science Algorithm Error: ", n_impl_impl/df_mlerr_labels.shape[0])

Environment and settings:  0.05227882037533512
Processing:  0.022788203753351208
Implementation:  0.24396782841823056
    Implementation - Data Science lib wrong usage:  0.1675603217158177
    Implementation - Data Science Algorithm Error:  0.07640750670241286


In [26]:
df_mlerr_labels_sum["label_root_cause"].value_counts()/df_mlerr_labels_sum.shape[0]

label_root_cause
API misuse               0.209115
NB specific              0.194370
implementation error     0.167560
environment setting      0.167560
data confusion           0.164879
unknown                  0.038874
insufficient resource    0.022788
ML model confusion       0.022788
library cause            0.012064
Name: count, dtype: float64

In [28]:
# other implementation:
1-0.052-0.023-0.244

0.6809999999999999